In [3]:
import pandas as pd
import random
from tqdm import tqdm
import os
import string
import pickle

In [2]:
import os

In [3]:
import string

In [4]:
import pickle

In [131]:
string.ascii_lowercase

'abcdefghijklmnopqrstuvwxyz'

In [9]:
df = pd.read_csv('5_letters.csv')
df = pd.DataFrame(df)
df.head()

,1,2,3,4,5
0,a,b,a,c,k
1,a,b,a,s,e
2,a,b,a,t,e
3,a,b,a,y,a
4,a,b,b,e,y


In [44]:
df.apply(lambda row: 'a' in row[[str(i) for i in range(1, 6)]].values, axis=1)

0        True
1        True
2        True
3        True
4        True
        ...  
2494    False
2495    False
2496    False
2497    False
2498    False
Length: 2499, dtype: bool

In [45]:
~df.apply(lambda row: 'a' in row[[str(i) for i in range(1, 6)]].values, axis=1)

0       False
1       False
2       False
3       False
4       False
        ...  
2494     True
2495     True
2496     True
2497     True
2498     True
Length: 2499, dtype: bool

In [46]:
df[df.apply(lambda row: 'a' in row[[str(i) for i in range(1, 6)]].values, axis=1)]


,1,2,3,4,5
0,a,b,a,c,k
1,a,b,a,s,e
2,a,b,a,t,e
3,a,b,a,y,a
4,a,b,b,e,y
...,...,...,...,...,...
2484,w,r,a,t,h
2489,y,a,c,h,t
2490,y,a,r,d,s
2491,y,e,a,r,s


In [47]:
df[df.apply(lambda row: 'a' in row[["1"]].values, axis=1)]

,1,2,3,4,5
0,a,b,a,c,k
1,a,b,a,s,e
2,a,b,a,t,e
3,a,b,a,y,a
4,a,b,b,e,y
...,...,...,...,...,...
238,a,x,l,e,s
239,a,x,o,n,s
240,a,z,i,d,e
241,a,z,o,l,e


In [48]:
df[df.apply(lambda row: tuple({"a"})[0] in row[["2"]].values, axis=1)]

,1,2,3,4,5
243,b,a,b,e,l
244,b,a,b,e,s
245,b,a,b,k,a
246,b,a,c,k,s
247,b,a,c,o,n
...,...,...,...,...,...
2441,w,a,t,e,r
2442,w,a,t,t,s
2443,w,a,v,e,s
2489,y,a,c,h,t


In [49]:
to_find = "aback"

In [50]:
first_word = 'slate'

In [132]:
def eval(word_to_find, 
         current_word):
    
    compo_word_to_find = list(word_to_find)
    sc = [0 for _ in range(len(word_to_find))]
    
    for i in range(len(current_word)):
        if current_word[i] == word_to_find[i]:
            compo_word_to_find.remove(current_word[i])
            sc[i] = 1
    
    for i in range(len(current_word)):
        if sc[i] == 0 and current_word[i] in compo_word_to_find:
            compo_word_to_find.remove(current_word[i])
            sc[i] = 0.5

    return tuple(sc)
    

In [52]:
eval("arbre", "kayak")

[0, 0.5, 0, 0, 0]

In [53]:
eval(to_find, first_word)

[0, 0, 1, 0, 0]

In [54]:
class Solver:
    def __init__(self, dataframePath = '5_letters.csv') -> None:
        
        self.df = df = pd.read_csv(dataframePath)
        self.df = pd.DataFrame(df)

        
        self.currentWord    = 'soare'
        self.current        = []
        self.infos          = dict()
        
        for i in range(5):
            self.infos[i] = {"found" : False, 
                                    "banned" : set(),
                                    "possibles" : set()}
        pass
    
    def updateInfos(self, evalResult):
        for i in range(len(evalResult)):
            match evalResult[i]:
                case 1:
                    self.infos[i]["found"]      = True
                    self.infos[i]["possibles"]  = {self.currentWord[i]}
                case 0.5:
                    for j in range(5):
                        if self.infos[j]["found"] == False and j != i:
                            self.infos[j]["possibles"].add(self.currentWord[i])
                    if self.currentWord[i] in self.infos[i]["possibles"]:
                        self.infos[i]["possibles"].remove(self.currentWord[i])
                case 0:
                    for j in range(5):
                        if self.infos[i]["found"] == False and self.currentWord[i] in self.infos[i]["possibles"]:
                            self.infos[i]["possibles"].remove(self.currentWord[i])
                            self.infos[i]["banned"].add(self.currentWord[i])
    
    def nextGuess(self) -> str:
        for i in range(5):
            if self.infos[i]["found"]:
                self.df = self.df[self.df.apply(lambda row: tuple(self.infos[i]["possibles"])[0] in row[[f"{i+1}"]].values, axis=1)]
                #print(self.infos)
                #print(self.df.head())
                continue
            else:
                pass
                # for j in self.infos[i]["banned"].copy():
                #     self.df = self.df[~self.df.apply(lambda row: j in row[[f"{i+1}"]].values, axis=1)]
                #     self.infos[i]["banned"].remove(j)
        self.currentWord = ''.join(self.df.iloc[random.randint(0, self.df.shape[0]-1)].values)
        return self.currentWord

In [55]:
len(set([1, 2, 3]).intersection(set([2, 4, 5]))), set([1, 2, 3]).intersection(set([2, 4, 5]))

(1, {2})

In [56]:
len(df)

2499

In [57]:
def meanFound(to_find, model, maxTries = 200, meanOver = 100):
    to_find = to_find
    ans = []
    for _ in tqdm(range(meanOver), desc="Calculatinf the mean of the model"):
        solver = Solver()
        tries = 0
        for i in range(maxTries):
            tries += 1
            if solver.currentWord == to_find:
                break
            else:
                solver.currentWord = solver.nextGuess()
                solver.updateInfos(eval(to_find, solver.currentWord))
        ans.append(tries)
    return sum(ans)/len(ans)
    

In [58]:
#meanFound("aback", Solver())

In [59]:
to_find = "aback"
bob = Solver()
for i in range(20):
    resultEval = eval(to_find, bob.currentWord)
    print(f"Guess {i+1} ; word: {bob.currentWord} | eval : {resultEval}")
    if resultEval == [1, 1, 1, 1, 1]:
        print(f"Found in {i+1} guesses")
        break
    bob.updateInfos(resultEval)
    bob.nextGuess()
    

Guess 1 ; word: soare | eval : [0, 0, 1, 0, 0]
Guess 2 ; word: trash | eval : [0, 0, 1, 0, 0]
Guess 3 ; word: spare | eval : [0, 0, 1, 0, 0]
Guess 4 ; word: enact | eval : [0, 0, 1, 1, 0]
Guess 5 ; word: place | eval : [0, 0, 1, 1, 0]
Guess 6 ; word: crack | eval : [0, 0, 1, 1, 1]
Guess 7 ; word: aback | eval : [1, 1, 1, 1, 1]
Found in 7 guesses


## Second version

In [113]:
set([1, 2, 3]) - set([2, 4, 7])

{1, 3}

In [119]:
class SolverBoy:
    def __init__(self, dataframePath = '5_letters.csv', connexionsPath = None) -> None:
        self.dfPath = dataframePath
        
        self.df             = pd.read_csv(dataframePath)
        self.df             = pd.DataFrame(df)

        self.countLetters   = dict([(i, {"nb": 0, 
                                         "maxtested" : 0,
                                         "possPlacement" : [None for _ in range(5)]}) for i in string.ascii_lowercase])
        
        temp                = self.df.iloc[:, 0].values
        
        for i in range(1, len(self.df.columns)):
            temp = temp + self.df.iloc[:, i].values
        
        self.words          = set(temp)
        self.currentWord    = 'soare'
        self.possibleWords  = self.words.copy()
        self.lastWord       = None 
        
        self.contains       = set()
        self.truePlaced     = [None for _ in range(5)]
        self.falsePlaced    = [set() for _ in range(5)]
        
        self.connexions     = self._getWordConnexions(connexionsPath)
        
        print(f"Loaded {len(self.words)} words, {tuple(self.words)[:10]}")
    
    def analyseResult(self, resultEval):
        
        for i in range(len(resultEval)):
            match resultEval[i]:
                case 1:
                    self.truePlaced[i] = self.currentWord[i]
                    self.countLetters[self.currentWord[i]]["possPlacement"][i] = 1
                case 0.5:
                    
                    self.falsePlaced[i].add(self.currentWord[i])
                    self.countLetters[self.currentWord[i]]["possPlacement"][i] = 0
                        
                case 0:
                    self.falsePlaced[i].add(self.currentWord[i])
                    self.countLetters[self.currentWord[i]]["possPlacement"][i] = 0
                    pass
                
        for l in tuple(set(self.currentWord)):
            if self.currentWord.count(l) >= self.countLetters[l]["maxtested"]:
                self.countLetters[l]["maxtested"]   = self.currentWord.count(l)
                self.countLetters[l]["nb"]          = sum([1 if self.currentWord[i] == l and resultEval[i] != 0 else 0 for i in range(5)])
                
                #we update all the possible positions of that letter
                for i in range(5):
                    if (l not in self.falsePlaced[i] == l) and l != self.truePlaced[i]:
                        self.countLetters[l]["possPlacement"][i] = 0.5
        #print(self.connexions[self.currentWord].keys())
        print(self.possibleWords)
        print(self.currentWord, resultEval, set(self.connexions[self.currentWord][tuple(resultEval)]))
        self.possibleWords = set(self.connexions[self.currentWord][tuple(resultEval)]).intersection(self.possibleWords).union(self.possibleWords - set([[].extend(self.connexions[self.currentWord][k]) for k in self.connexions[self.currentWord].keys()]))
        
        
    def _allPossibleEvals(self):
        return [[a, b, c, d, e] for a in [0, 0.5, 1] for b in [0, 0.5, 1] for c in [0, 0.5, 1] for d in [0, 0.5, 1] for e in [0, 0.5, 1]]
    
    def _eval(self, word_to_find, 
         current_word):
    
        compo_word_to_find = list(word_to_find)
        sc = [0 for _ in range(len(word_to_find))]
        
        for i in range(len(current_word)):
            if current_word[i] == word_to_find[i]:
                compo_word_to_find.remove(current_word[i])
                sc[i] = 1
        
        for i in range(len(current_word)):
            if sc[i] == 0 and current_word[i] in compo_word_to_find:
                compo_word_to_find.remove(current_word[i])
                sc[i] = 0.5

        return sc
    
    def _getWordConnexions(self, savePath = None):
        if savePath == None:
            savePath = self.dfPath[:-4] + "_connexions.pkl"
        if os.path.exists(savePath):
            with open(savePath, 'rb') as f:
                return pickle.load(f)
            
        else:
            self.connexions = dict([(w, {}) for w in self.words])
            bar = tqdm(total=pow(len(self.words), 2), desc="Generating connexions")
            for w in self.connexions.keys():
                for w2 in self.words:
                    evalResult = tuple(self._eval(w, w2))
                    if evalResult in self.connexions[w].keys():
                        self.connexions[w][evalResult].append(w2)
                    else:
                        self.connexions[w][evalResult] = [w2]
                    bar.update(1)
            bar.close()

            #save the connexions in a pkl file
            with open(savePath, 'wb') as f:
                pickle.dump(self.connexions, f)
            
            return self.connexions
        
    def getNextWord(self):
        self.lastWord = self.currentWord
        scores = dict()
        #scores = dict([(w, float('inf')) for w in self.words])
        for w in self.connexions.keys():
            temp = [len(set(self.connexions[w][i]).intersection(set(self.possibleWords))) for i in self.connexions[w].keys() if len(set(self.connexions[w][i]).intersection(set(self.possibleWords)))]
            #scores[w] = sum(temp)/len(temp) if len(temp) else float('inf')
            scores[(sum(temp)/len(temp)) -0.2*sum(temp) if len(temp) else float('inf')] = w
        
        # the score key with the minimum value
        
        self.currentWord = scores[min(scores.keys())]
        print(scores)
        # print(self.connexions[self.currentWord].values())
        return self.currentWord
        
            

In [61]:
sum([1 if "kayak"[i] == "k" and [1, 0.5, 0.5, 0.5, 0.5][i] != 0 else 0 for i in range(5)])

2

In [64]:
bob = SolverBoy()

Loaded 2499 words, ('ahead', 'tears', 'bunch', 'under', 'diner', 'barge', 'exile', 'dwell', 'bipod', 'wider')


In [ ]:
#bob.differenciateWords()

In [ ]:
eval("aback", "kayak")

[0, 0.5, 0, 0.5, 1]

In [101]:
bob.currentWord = "kayak"
bob.analyseResult(eval("aback", bob.currentWord))

KeyError: 'kayak'

In [ ]:
bob.falsePlaced, bob.truePlaced

([{'k'}, {'a'}, {'y'}, {'a'}, set()], [None, None, None, None, 'k'])

In [120]:
bob = SolverBoy()
to_find = random.choice(tuple(bob.words))
print(f"to find: {to_find}")

for i in range(20):
    bob.currentWord = bob.getNextWord()
    print(f"Guess {i+1} ; word: {bob.currentWord}")
    if bob.currentWord == to_find:
        print(f"Found in {i+1} guesses")
        break
    bob.analyseResult(eval(to_find, bob.currentWord))
    #bob.nextGuess()

Loaded 2499 words, ('ahead', 'tears', 'bunch', 'under', 'diner', 'barge', 'exile', 'dwell', 'bipod', 'wider')
to find: flour
{-479.4829268292683: 'penis', -465.5671232876712: 'holly', -478.25689655172414: 'adder', -475.05742574257425: 'aorta', -458.15000000000003: 'boozy', -478.8: 'apses', -476.0: 'cowed', -472.9290322580645: 'pound', -477.6849557522124: 'shake', -473.76875: 'fazes', -480.4279069767442: 'caned', -466.02972972972975: 'going', -476.2245283018868: 'braze', -469.69156626506026: 'glory', -472.63695652173914: 'bible', -474.3: 'amiss', -480.1228346456693: 'traps', -475.7711538461539: 'threw', -472.03333333333336: 'poker', -460.1333333333333: 'cubic', -479.808: 'surge', -463.5826086956522: 'doggy', -467.76153846153846: 'crumb', -471.40227272727276: 'risky', -473.4947368421053: 'duped', -481.425: 'doers', -478.0695652173913: 'acted', -478.44102564102565: 'copra', -475.3: 'grave', -479.9666666666667: 'shore', -477.4875: 'ditsy', -477.2864864864865: 'devil', -471.72134831460676: 

In [4]:
class SolverBoy:
    def __init__(self, dataPath = 'updated5letters.txt', firstWord = None, lenWords = 5) -> None:
        self.countLetters   = dict([(i, {"nb": 0, 
                                            "maxtested" : 0,
                                            "possPlacement" : [None for _ in range(lenWords)]}) for i in string.ascii_lowercase])
        if dataPath.endswith('.csv'):
            self.dfPath = dataPath
        
            self.df             = pd.read_csv(dataPath)
            self.df             = pd.DataFrame(df)
            
            temp                = self.df.iloc[:, 0].values
            
            for i in range(1, len(self.df.columns)):
                temp = temp + self.df.iloc[:, i].values
                
        elif dataPath.endswith('.txt'):
            with open(dataPath, 'r') as f:
                temp = [i.strip().lower() for i in f.readlines() if len(i.strip()) == lenWords]
            
        else:
            raise ValueError("The dataPath should be a .csv or .txt file")
            
        self.firstWord      = firstWord
        
        self.words          = set(temp)
        self.currentWord    = 'soare'
        self.possibleWords  = self.words.copy()
        self.possibleOuptus = dict()
        self.lastWord       = None 
    
    def analyseResult(self, resultEval):
        if resultEval == (1, 1, 1, 1, 1):
            return None
        self.possibleWords = self.possibleOuptus[tuple(resultEval)]
        self.lastWord = self.currentWord
        
    def analyseResults(self, resultEvals):
        resultEvals = dict(resultEvals) if type(resultEvals) != dict else resultEvals
        
        for word in resultEvals.keys():
            if resultEval[word] == tuple([1 for _ in range(len(word))]):
                self.currentWord = word
                return None

            self.lastWord = self.currentWord
            self.currentWord = word
            self.possibleOuptus = self._getPossibilities(self.currentWord)
            self.possibleWords = self.possibleOuptus[tuple(resultEvals[word])]

    def _eval(self, word_to_find, 
         current_word):
    
        compo_word_to_find = list(word_to_find)
        sc = [0 for _ in range(len(word_to_find))]
        
        for i in range(len(current_word)):
            if current_word[i] == word_to_find[i]:
                compo_word_to_find.remove(current_word[i])
                sc[i] = 1
        
        for i in range(len(current_word)):
            if sc[i] == 0 and current_word[i] in compo_word_to_find:
                compo_word_to_find.remove(current_word[i])
                sc[i] = 0.5

        return tuple(sc)
    
    def _getPossibilities(self, pivot):
        possibilities = dict()
        for word in self.possibleWords:
            res = self._eval(word, pivot)
            if res in possibilities:
                possibilities[res].append(word)
            else:
                possibilities[res] = [word]
        return possibilities
        
    def getNextWord(self):
        
        #for the fist word
        if type(self.firstWord) == str and (len(self.words) == len(self.possibleWords)):
            possibilities = self._getPossibilities(self.firstWord)
            self.currentWord = self.firstWord  if len(possibilities.keys()) > 1 else possibilities[list(possibilities.keys())[0]][0]
            self.possibleOuptus = possibilities
            return self.firstWord
        
        #for every other word
        ans = {
            "pivot" : None,
            "possibilities" : dict()
        }
        for pivot in self.words:
            possibilities = self._getPossibilities(pivot)
            if len(possibilities) > len(ans["possibilities"].keys()):
                ans["pivot"] = pivot
                ans["possibilities"] = possibilities

        self.currentWord = ans["pivot"] if len(ans["possibilities"].keys()) > 1 else ans["possibilities"][list(ans["possibilities"].keys())[0]][0]
        self.possibleOuptus = ans["possibilities"]
        
        return self.currentWord


In [171]:
to_find = "aback"
bob = SolverBoy(firstWord="cares")
for i in range(20):
    resultEval = eval(to_find, bob.getNextWord())
    print(f"Guess {i+1} ; word: {bob.currentWord} | eval : {resultEval}")
    if resultEval == (1, 1, 1, 1, 1):
        print(f"Found in {i+1} guesses")
        break
    bob.analyseResult(resultEval)

Guess 1 ; word: cares | eval : (0.5, 0.5, 0, 0, 0)
Guess 2 ; word: litho | eval : (0, 0, 0, 0, 0)
Guess 3 ; word: rumba | eval : (0, 0, 0, 0.5, 0.5)
Guess 4 ; word: aback | eval : (1, 1, 1, 1, 1)
Found in 4 guesses


In [172]:
def evaluateModel(iters = 100):
    
    score = 0
    for _ in tqdm(range(iters), desc="Evaluating the model"):
        bob = SolverBoy(firstWord="tares")
        to_find = random.choice(tuple(bob.words))
        to_find = "aback"
        for i in range(1, 7):
            resultEval = eval(to_find, bob.getNextWord())
            if resultEval == (1, 1, 1, 1, 1):
                score += i
                break
            bob.analyseResult(resultEval)
        if i == 6:
            pass
    return score/iters

In [174]:
#print("Average number of tries to find the word: {:2} tries".format(evaluateModel(10_000)))

In [159]:
def playOnce(wordToFind = None, wordToStart = None, display = True):
    bob = SolverBoy(firstWord=wordToStart)
    to_find = wordToFind if wordToFind != None else random.choice(tuple(bob.words))
    if display:
        print(f"-> to find: {to_find}")
    for i in range(20):
        resultEval = eval(to_find, bob.getNextWord())
        if display:
            print(f"Guess {i+1} ; word: {bob.currentWord} | eval : {resultEval}")
        if resultEval == (1, 1, 1, 1, 1):
            if display:
                print(f"Found in {i+1} guesses")
            break
        bob.analyseResult(resultEval)

In [ ]:
#try with voice

In [145]:
eval("voice", "twice")

(0, 0, 1, 1, 1)

In [203]:
playOnce(wordToStart="cares")

-> to find: chore
Guess 1 ; word: cares | eval : (1, 0, 0.5, 0.5, 0)
Guess 2 ; word: trend | eval : (0, 0.5, 0.5, 0, 0)
Guess 3 ; word: chore | eval : (1, 1, 1, 1, 1)
Found in 3 guesses


In [49]:
bob = SolverBoy(firstWord='tares')
bob.getNextWord()

'tares'

In [50]:
alreadyDone = [
    ["tares", (0, 0, 0.5, 0.5, 0)],
    ["doing", (0, 0.5, 0, 0.5, 0)],
    ["crook", (0, 1, 0, 1, 0)],
    ["hydro", (0, 0, 0, 0.5, 0.5)],
    ["honor", (0, 0, 0.5, 1, 0.5)],
    ["slags", (0, 0, 0, 0, 0)],
    ["filet", (1, 0, 0, 0.5, 0)],
    ["humph", (0, 0, 0, 0, 0)],
    ["mince", (0, 0, 0.5, 0, 0.5)],
    ["skull", (0, 0, 0, 0, 0)],
    ["felon", (1, 0.5, 0, 1, 1)],
    ["crone", (0, 1, 0.5, 0.5, 0.5)],
    ["stark", (0, 0, 0, 0.5, 0)],
    ["resew", (0.5, 0.5, 0, 0, 0)],
    ["rider", (0.5, 0, 0, 0.5, 0)]
]
bob.analyseResults(alreadyDone)
bob.getNextWord()

KeyError: (0, 0.5, 0, 0.5, 0)

In [46]:
bob.analyseResult((1, 0, 0, 1, 0))
bob.getNextWord()

'rider'